In [ ]:
%%bash
mkdir ~/.kaggle
mv kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json
cat ~/.kaggle/kaggle.json #credenciales de kaggle generadas con la API

In [ ]:
p_base= !kaggle datasets download -d jerzydziewierz/bee-vs-wasp #bajamos de kaggle los datasets de imagenes

In [ ]:
!unzip bee-vs-wasp.zip 

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

In [ ]:
labels = pd.read_csv("kaggle_bee_vs_wasp/labels.csv", dtype=str)
labels

In [ ]:
duplicados = labels[labels.duplicated()]
print(duplicados) #Chequeamos no tener valores duplicados. No tenemos

In [ ]:
import os
labels.path = labels["path"].str.replace("\\","/")
labels.describe()
#reemplazamos la doble barra en el nombre de archivos del dataset de etiquetas porque genera problemas para que flow from dataframe detecte el path

In [ ]:
labels.shape

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
sns.set(style='white',context='notebook')
sns.countplot(labels['label'],palette='viridis');

plt.title('clases'.title(),fontsize=15);
plt.ylabel('count'.title(), fontsize=8)
plt.xlabel('seccion'.title(), fontsize=8)

In [ ]:
labels['label'].value_counts(1)

In [ ]:
#dropeamos other y nos quedamos con 3 clases
labels.drop(labels.loc[labels['label']=='other'].index, inplace=True)

In [ ]:
labels['label'].value_counts(1)

In [ ]:
from sklearn.model_selection import train_test_split

X=labels.drop(columns=['label','id'])
y=labels["label"]

train, val = train_test_split(labels, test_size=.2, random_state=42)
val, test = train_test_split(val, test_size=.5, random_state=42)
print('train:', train.shape, '\nval:', val.shape, '\ntest',  test.shape)

In [ ]:
datagen = ImageDataGenerator(rescale=1./255.,validation_split=0.25)
        # Train 
train_g = datagen.flow_from_dataframe(train,
            directory="kaggle_bee_vs_wasp/",
            x_col="path",
            y_col="label",
            has_ext=False,
            subset="training",
            batch_size=64,
            classes=None,
            seed=42,
            shuffle=True,
            class_mode="categorical",
            target_size=(150, 150))


        # Validation generator
val_g = datagen.flow_from_dataframe(val,
            directory="kaggle_bee_vs_wasp/",
            x_col="path",
            y_col="label",
            has_ext=False,
            subset="validation",
            batch_size=64,
            seed=42,
            classes=None,
            shuffle=True,
            class_mode="categorical",
            target_size=(150, 150))    

        # Test
test_datagen=ImageDataGenerator(rescale=1./255.)
test_g = test_datagen.flow_from_dataframe(test,
            directory="kaggle_bee_vs_wasp/",
            x_col="path",
            y_col='label',
            has_ext=False,
            target_size=(150, 150),
            class_mode="categorical",
            classes=None,
            batch_size=64,
            color_mode="rgb",
            seed=42,
            shuffle=False)
print(len(train))

In [ ]:
print('Total de imágenes en train:', len((val_g)))                                            

In [ ]:
print('Total de imágenes en test:', len((test_g)))

In [ ]:
#chequeamos valores que tenemos en train
labels["label"].value_counts()

In [ ]:
classes = list(train_g.class_indices.keys())
classes
w = 10
h = 10
fig = plt.figure(figsize=(10, 15))
columns = 4
rows = 8
ax = []
x,y = train_g.next()
for i in range(columns*rows):
    image = x[i]
    label = classes[list(y[i]).index(1)]
    ax.append( fig.add_subplot(rows, columns, i+1) )
    ax[-1].set_title(label)
    plt.imshow(image)
plt.show()

In [ ]:
classes

In [ ]:
#generador de imagenes nuevas
directory_path=None
batch_size=64
generador_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = generador_train.flow_from_dataframe(
            train,
            directory="kaggle_bee_vs_wasp/",
            x_col="path",
            y_col="label",
            has_ext=False,
            subset="training",
            batch_size=64,
            seed=42,
            clases=None,
            shuffle=True,
            class_mode="categorical",
            target_size=(150, 150),
            color_mode="rgb")

validation_generator = val_datagen.flow_from_dataframe(
            val,
            directory="kaggle_bee_vs_wasp/",
            x_col="path",
            y_col="label",
            has_ext=False,
            batch_size=64,
            seed=42,
            classes=None,
            shuffle=False,
            class_mode="categorical",
            target_size=(150, 150),
            color_mode="rgb") 


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
from google.colab import files
from keras.models import load_model
checkpoint_model3 = ModelCheckpoint('from_scratch/weights.{epoch:02d}-{val_loss:.2f}.h5', monitor='val_accuracy', verbose=0,
                             save_best_only=False, save_weights_only=False, mode='auto')

#files.download('model3.h5')


callbacks=[checkpoint_model3]


In [ ]:
#Probamos una red con data augmentation + regularización y dropout en la última capa
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
from keras import layers
from keras import models
from keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

dropout_rate=0.3
model3 = models.Sequential()
model3.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Conv2D(64, (3, 3), activation='relu'))
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Conv2D(128, (3, 3), activation='relu'))
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Conv2D(128, (3, 3), activation='relu'))
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Flatten())
model3.add(layers.Dense(512, activation='relu',kernel_regularizer=regularizers.l1(0.001)))
model3.add(Dropout(dropout_rate))
model3.add(layers.Dense(3, activation='softmax'))


In [ ]:
model3.summary()
model3.save('model3.h5')

In [ ]:
model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
#Antes de fitear vamos a equilibrar las clases 
from sklearn import preprocessing
lb_classes = preprocessing.LabelBinarizer()

# Ajustamos a la variable labels
lb_classes.fit(labels['label'])
lb_classes.classes_

In [ ]:
# Encodeamos las labels
labels_ohe = pd.DataFrame(lb_classes.transform(labels['label']), columns=lb_classes.classes_, index=labels['id'])
labels_ohe.head()

In [ ]:
class_weight = {k:v for k,v in zip(range(len(lb_classes.classes_)),
                                   (1/labels_ohe.iloc[train.index].sum()) * 100)}

class_weight

In [ ]:
# Visualizamos los pesos definidos
((1/labels_ohe.iloc[train.index].sum()) * 100).plot(kind='bar');

In [ ]:
history3 = model3.fit(
      train_generator,
      steps_per_epoch=train_generator.n//train_generator.batch_size, #train / batch size. 
      epochs=85,
      validation_data=validation_generator,
      validation_steps=validation_generator.n//validation_generator.batch_size,
      class_weight=class_weight,
      callbacks=callbacks)


In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model3, show_shapes=True)

In [ ]:
files.download('model3.h5')

In [ ]:
from keras.models import load_model
model3.save('model3.h5')
#model3 = load_model('model3.h5')

In [ ]:
acc3=history3.history['accuracy']
val_loss3=history3.history['val_loss']
loss3=history3.history['loss']
val_acc3=history3.history['val_accuracy']
epochs3 = range(1, len(acc3) + 1)

plt.figure(figsize=(25,8))
plt.title('Modelo de base')
plt.plot(epochs3,loss3)
plt.plot(epochs3,val_loss3)
plt.xticks(ticks=epochs3)
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Training loss','Validation Loss'])

plt.figure(figsize=(25,8))
plt.plot(epochs3,acc3)
plt.plot(epochs3,val_acc3)
plt.xticks(ticks=epochs3)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training Accuracy','Validation Accuracy']);

In [ ]:
# Predecimos
val_pred = model3.predict(validation_generator)

# Obtenemos la matriz de etiquetas reales de validación
val_labels = lb_classes.transform(val['label'])

# Evaluamos las shapes de los arrays resultantes
val_pred.shape, val_labels.shape

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Definimos listas vacías para ir volcando las métricas por clase
accuracy_per_class = []
precision_per_class = []
recall_per_class = []
f1_per_class = []

# Definimos un listado de etiquetas de clase
classes = list(lb_classes.classes_)

for i in range(val_labels.shape[1]):
    # Computamos las métricas de evaluación por clase y hacemos un append a la lista correspondiente
    accuracy_per_class.append(accuracy_score(val_labels[:, i], (val_pred[:, i] > 0.5).astype(int)).round(2))
    precision_per_class.append(precision_score(val_labels[:,  i], (val_pred[:, i] > 0.5).astype(int)).round(2))
    recall_per_class.append(recall_score(val_labels[:,  i], (val_pred[:, i] > 0.5).astype(int)).round(2))
    f1_per_class.append(f1_score(val_labels[:,  i], (val_pred[:, i] > 0.5).astype(int)).round(2))
    
    print(classes[i])
    print('Accuracy:', accuracy_per_class[i], 'Precision:', precision_per_class[i],
          'Recall:', recall_per_class[i], 'F1:', f1_per_class[i])
    
    # Visualizamos la matriz de confusión por clase
    cm  = confusion_matrix(val_labels[:,  i], (val_pred[:, i] > 0.5).astype(int))
    sns.heatmap(cm, annot=True, cmap='Purples', fmt='.0f', cbar=False, square=True,)
    plt.ylabel('Etiqueta real')
    plt.xlabel('Etiqueta predicha')
    plt.title('Confusion Matrix')
    plt.show()

In [ ]:
model3.metrics_names

In [ ]:
#generar best_threshold
threshold = np.arange(0.1, 0.9, 0.05)

# Definimos listas vacíos donde iremos volcando los resultados
acc_ = []
accs = []
best_threshold = np.zeros(val_pred.shape[1])

# Iteramos sobre las distintas clases
for i in range(len(best_threshold)):
    # Indexamos las probabilidades predichas por clase
    y_prob = np.array(val_pred[:, i])
    
    # Iteramos sobre los posibles umbrales
    for j in threshold:
        # Binarizamos las predicciones de acuerdo al umbral
        y_pred = [1 if prob >= j else 0 for prob in y_prob]
        y_pred = np.array(y_pred)
        # Calculamos accuracy
        acc = accuracy_score(val_labels[:, i], y_pred)
        acc_.append(acc)
    
    # Volcamos los resultados en las listas generales
    acc_  = np.array(acc_)
    index = np.where(acc_==acc_.max())
    accs.append(acc_.max())
    best_threshold[i] = threshold[index[0][0]]
    # Vaciamos la lista de para volver a iterar
    acc_ = []

In [ ]:
accs

In [ ]:
best_threshold

In [ ]:
# Tomamos estos umbrales para generar las clasificaciones
y_pred = np.array([[1 if val_pred[i, j] >= best_threshold[j] else 0 for j in range(val_labels.shape[1])] for i in range(len(val_labels))])

In [ ]:
model3.evaluate(test_g)

In [ ]:
#vemos como generaliza el modelo
# Predecimos
test_pred = model3.predict(test_g)

# Tomamos los umbrales óptimos obtenidos en validación para mapear probabilidades a etiquetas de clase
y_pred = np.array([[1 if test_pred[i, j] >= best_threshold[j] else 0 for j in range(test_pred.shape[1])] for i in range(len(test_pred))])

# Creamos un DF para facilitar luego la visualización de las predicciones
y_pred = pd.DataFrame(y_pred, index=test.index)

# Evaluamos las shapes de los arrays resultantes
y_pred.shape

In [ ]:
import time
from IPython.display import clear_output
from tensorflow.keras.preprocessing.image import load_img, img_to_array
data_dir= "/content/kaggle_bee_vs_wasp"

n = 50
bichos = list()

for i in range(n):
    random_bichos = np.random.choice(test['path'])
    while random_bichos in bichos:
        random_bichos = np.random.choice(test['path'])
    bichos.append(random_bichos)

    # Imagen original
    random_bichos_path = os.path.join(data_dir, random_bichos)
    test_image = load_img(random_bichos_path, target_size=(224, 224))
    test_image = img_to_array(test_image)
 
    # Predicción
    prediction = y_pred.loc[test.loc[test['path'] == random_bichos].index[0]]
    predicted_labels = (prediction > 0.5).astype(int)

    # Etiquetas
    labels = lb_classes.classes_[predicted_labels > 0]

    # Visualizamos la imagen y su predicción
    title = "{}\n Etiquetas reales = {}\nEtiquetas predichas = {}" \
                .format(test.loc[test['path'] == random_bichos, 'path'].values[0],
                        test.loc[test['path'] == random_bichos, 'label'].values[0],
                        labels)

    plt.imshow(plt.imread(os.path.join(data_dir, random_bichos)))
    plt.title(title)
    plt.axis('off')
    plt.show();
    
    if i > 0 and i % 10 == 0:
        time.sleep(20)
        clear_output(wait=True)